**Validando o período de treinamento**

O periodo utilizado para treinar o modelo é um parâmetro importante, o contexto do mercado financeiro no muda a todo momento e os parâmetros do nosso modelo devem ser atualizados.

**Objetivos:**  
- Treinar modelos para 1, 2, 3 e 5 anos de dados históricos e verificar variações no desempenho sem cross validation (5 ativos distintos para cada);
- Terinar modelos utilizando Blocking Time Series Split e Time Series Split.

**Modelo utilizado:**
- Proporção de treino e teste -> 0.7, 0.3
- Árvore de Decisão com max_depth = 5 e random_state = 42

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
import warnings

import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

import aurum

from sklearn.tree import DecisionTreeClassifier

warnings.filterwarnings('ignore')

In [2]:
# Funções Helpers:

class BlockingTimeSeriesSplit():
    """
    Created on Mon Jul 29 16:00:33 2019
    @author: TNDUser
    """
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]


def get_ohlcv_adapted(TICKER, DIST_ALVO=5, PERIODO=1):
    """_summary_

    Args:
        TICKER (str): Código do ativo
        DIST_ALVO (int, optional): Distancia em dias do alvo. Defaults to 5.
        PERIODO (int, optional): Número de anos usados trazidos. Defaults to 1.

    Returns:
        _type_: _description_
    """

    ticker = yf.Ticker(TICKER) 

    END_DATE = '2022-06-01'
    START_DATE = pd.to_datetime(END_DATE) - pd.DateOffset(365 * PERIODO)

    df = ticker.history(
        start=START_DATE,
        end=END_DATE,
        interval='1d',
    ).reset_index()

    assert DIST_ALVO > 0, 'O número de dias paro o alvo precisa ser maior ou igual a zero.'

    df.loc[:, 'LEAK_Retorno'] = (df['Close'].shift(-DIST_ALVO) - df['Close'])/df['Close']
    df.loc[:, 'Alvo'] = (df['LEAK_Retorno'] > 0.00).astype('int')


    return df


def treinar_modelo(TICKER, PERIODO):
    
    import warnings
    warnings.filterwarnings('ignore')

    df = get_ohlcv_adapted(TICKER, PERIODO)
    df = aurum.ft.technical_indicators(df)

    FEATURES = [
        'RSI_14',
        'STOCHd_14_3_3',
        'ROC_2',
        'ROC_5',
        'ROC_10',
        'SLOPE_3',
        'ATRr_5',
        'WILLR_14',
        'OBV_ROC_14',
        'EMA_BUY_CROSS',
        'EMA_SELL_CROSS',
        'EMA_9_DISTANCE',
        'EMA_21_DISTANCE',
        'BBAND_FechouFora_Lower',
        'BBAND_FechouFora_Upper'
    ]

    TARGET = ['Alvo']

    train_size = int(len(df) * .7)

    X_train = df.loc[:train_size, FEATURES]
    y_train = df.loc[:train_size, TARGET]

    X_test = df.loc[train_size:, FEATURES]
    y_test = df.loc[train_size:, TARGET]

    dtc = DecisionTreeClassifier(
        max_depth=5,
        random_state=42,
    )

    dtc.fit(X_train, y_train)

    y_pred = [1 if x == True else 0 for x in (dtc.predict_proba(X_test)[:, 1] > .5)]

    resultados = df[train_size:].reset_index(drop=True)
    resultados['PREDICOES'] = y_pred
    resultados['RETORNO_MODELO'] = 1 + ((resultados['LEAK_Retorno']/5)* resultados['PREDICOES'])
    resultados['RETORNO_BNH'] = 1 + ((resultados['LEAK_Retorno']/5))
    resultados['RETORNO_ACUMULADO_MODELO'] = resultados['RETORNO_MODELO'].cumprod()
    resultados['RETORNO_ACUMULADO_BNH'] = 1 + (df['Close'] - df.loc[0, 'Close'])/df.loc[0, 'Close']


    retorno = pd.DataFrame(columns=['Ticker', 'Periodo', 'RetornoModelo', 'RetornoBnH'])
    retorno.loc[0, 'Ticker'] = TICKER
    retorno.loc[0, 'Periodo'] = PERIODO
    retorno.loc[0, 'RetornoModelo'] = resultados.RETORNO_ACUMULADO_MODELO.tail(1).values[0]
    retorno.loc[0, 'RetornoBnH'] = resultados.RETORNO_ACUMULADO_BNH.tail(1).values[0]


    return retorno

def full_test(LISTA_TICKERS):

    resultados = pd.DataFrame()
    for TICKER in LISTA_TICKERS:
        for PERIODO in [1, 2, 3, 5]:
            resultados = pd.concat([resultados, treinar_modelo(TICKER, PERIODO)])

    return resultados

In [3]:
LISTA_ATIVOS = [
    'ABEV3.SA',
    'BBAS3.SA',
    'BEEF3.SA',
    'CMIG4.SA',
    'CSAN3.SA',
    'EQTL3.SA',
    'GGBR4.SA',
    'ITSA4.SA',
    'ITUB4.SA',
    'PETR3.SA',
    'RADL3.SA',
    'SANB11.SA',
    'SUZB3.SA',
    'TAEE11.SA',
    'VALE3.SA',
    'WEGE3.SA'
]

teste_periodo = full_test(LISTA_ATIVOS)

In [6]:
teste_periodo.head()

,Ticker,Periodo,RetornoModelo,RetornoBnH
0,ABEV3.SA,1,1.012505,0.884593
0,ABEV3.SA,2,1.032585,0.884593
0,ABEV3.SA,3,0.972979,0.889279
0,ABEV3.SA,5,1.060023,0.889279
0,BBAS3.SA,1,0.982014,0.970888


In [5]:
teste_periodo.groupby('Periodo').mean()

,RetornoModelo,RetornoBnH
Periodo,,
1,1.003907,0.961149
2,1.019509,0.961149
3,1.014534,0.964257
5,1.062822,0.965497


In [10]:
teste_periodo[['Periodo','RetornoModelo', 'RetornoBnH']].groupby('Periodo').std()

,RetornoModelo,RetornoBnH
Periodo,,
1,0.027590,0.111398
2,0.043971,0.111398
3,0.067633,0.116383
5,0.122839,0.116828
